In [1]:
%matplotlib inline


Advanced: Making Dynamic Decisions and the Bi-LSTM CRF
======================================================

Dynamic versus Static Deep Learning Toolkits
--------------------------------------------



Implementation Notes
--------------------

The example below implements the forward algorithm in log space to
compute the partition function, and the viterbi algorithm to decode.
Backpropagation will compute the gradients automatically for us. We
don't have to do anything by hand.

The implementation is not optimized. If you understand what is going on,
you'll probably quickly see that iterating over the next tag in the
forward algorithm could probably be done in one big operation. I wanted
to code to be more readable. If you want to make the relevant change,
you could probably use this tagger for real tasks.



In [2]:
import sys
import torch
print("python version: %s" %(sys.version))
print("torch version: %s" %(torch.__version__))

python version: 3.7.7 (default, Mar 10 2020, 15:43:33) 
[Clang 11.0.0 (clang-1100.0.33.17)]
torch version: 1.5.0


In [3]:
# Author: Robert Guthrie

#导入pytorch相关的包
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

Helper functions to make the code more readable.



In [4]:

def argmax(vec):
    # 返回每一行最大值的索引（位置）
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):  # seq是字序列，to_ix是字和序号的字典
    idxs = [to_ix[w] for w in seq] # idxs是字序列对应的向量
    # 将句子转化为id
    return torch.tensor(idxs, dtype=torch.long)


# 对所有可能路径的分数取指数加和再取log
# 前向算法是不断积累之前的结果，缺点是指数和累积到一定程度后，会超过计算机浮点值的最大值，变成无限大
# 为了避免这种情况，用一个合适的值clip去提指数和的公因子，这样就不会使某项变得过大而无法计算
# SUM = log(exp(s1)+exp(s2)+...+exp(s100))
#    = log{(exp(clip)*[exp(s1-clip)+exp(s2-clip)+...+exp(s100-clip)]}
#    = clip+ log[exp(s1-clip)+exp(s2-clip)+...+exp(s100-clip)]
# where clip = max
# Compute log sum exp in a numerically stable way for the forward algorithm

def log_sum_exp(vec):   #vec.shape=[1, target_size]
    max_score = vec[0, argmax(vec)]# 每一行的最大值 
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1]) # 广播，扩展维度
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast))) # 所有vec减去最大值的广播值，避免某项过大无法计算


Create model



In [5]:
class BiLSTM_CRF(nn.Module):
    
    # 定义结构，初始化参数

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim # 词向量维度
        self.hidden_dim = hidden_dim       # Bi-LSTM隐层含维度
        self.vocab_size = vocab_size      # 字典中有多少字
        self.tag_to_ix = tag_to_ix        # tag转换成id
        self.tagset_size = len(tag_to_ix) # tag的数量

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim) #定义embedding表，是有vocab_size行，embedding_dim列矩阵
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True) # 输入是embedding_dim

        #将BiLSTM提取的特征向量映射到特征空间（经过全连接得到发射分数）
        # Maps the output of the LSTM into tag space. 
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)


        #转移矩阵的参数初始化，transitions{i,j}代表的是从第j个tag转移到第i个tag的转移分数
        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(                     # 随机初始化转移矩阵
            torch.randn(self.tagset_size, self.tagset_size)) # tagset_size*tagset_size的矩阵

        
        #初始化所有其他tag转移到START_TAG的分数非常小，即不可能由其他tag转移到START_TAG
        #初始化STOP_TAG转移到所有其他tag的分数非常小，即不可能由STOP_TAG转移到其他tag
        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000   # tag_to_ix[START_TAG]: 3（第三行，即其他状态到START_TAG的概率）
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000    # tag_to_ix[STOP_TAG]: 4（第四列，即STOP_TAG到其他状态的概率）

        self.hidden = self.init_hidden() # 初始化lstm参数

    def init_hidden(self):
        # 初始化LSTM参数
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        # 通过前向算法递推计算所有可能路径的分数总和（动态规划算法），CRF的分母
        init_alphas = torch.full((1, self.tagset_size), -10000.)  # 初始隐状态概率，第1个字是O1的实体标记是qi的概率
        # START_TAG has all of the score.
        # 初始化step 0 即START位置的发射分数，START_TAG取0其他位置取-10000（在序列开始只可能是START_TAG）
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        # 将初始化START位置为0的发射分数赋值给forward_var，随着step t变化
        forward_var = init_alphas

        # Iterate through the sentence
        # 迭代整个句子
        for feat in feats:  #对每个时间步循环，feat的维度是[1, target_size]
            # 当前时间步的前向tensor
            # The forward tensors at this timestep
            alphas_t = []  #存储当前时间步的对应每个tag分数
            
            # 给定每一帧的发射分值，按照当前的CRF层参数算出所有可能序列的分值和
            for next_tag in range(self.tagset_size):   # 当前的时间步可以转移到各个tag，分别计算各个tag分数
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                # 取出当前tag的发射分数，与之前时间步的tag无关
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                # 取出当前tag由之前tag转移过来的转移分数
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                # 当前路径分数：之前时间步分数 + 转移分数 + 发射分数（本身应该相乘求解的，因为用log计算，所以改为相加）
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                # 对当前分数取log-sum-exp
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            # forward_var 递推计算下一个时间步
            forward_var = torch.cat(alphas_t).view(1, -1)
        # 考虑最终转移到STOP_TAG，发射分值为0，转移分值为列向量 self.transitions[:, [self.tag2ix[END_TAG]]]
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        # 计算最终分数
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):#得到feats，维度=len(sentence)*tagset_size，表示句子中每个词是分别为target_size个tag的概率 
        # 通过Bi-LSTM提取特征
        self.hidden = self.init_hidden() # lstm参数初始化
        #词或字的序列转化成embedding的序列
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1) # 通过word_embeds查表，查找句子对应的embeddings
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)  #发射分数
        return lstm_feats

    def _score_sentence(self, feats, tags):  # 正确路径的分数，CRF的分子，输入是发射分数和路径的tags
        # Gives the score of a provided tag sequence
        # 计算给定tag序列的分数（一条路径的分数）
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            # 递推计算路径分数：转移分数 + 发射分数，self.transitions[tags[i + 1], tags[i]] 是从标签i到标签i+1的转移概率 
            # feat[tags[i+1]], feat是step i的输出结果，有５个值，对应B, I, E, START_TAG, END_TAG, 取对应标签的值
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]] # 沿途累加每一帧的转移和发射
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]] # 加上到END_TAG的转移
        return score

    def _viterbi_decode(self, feats): # 解码，得到预测序列的得分，以及预测的序列
        # 与_forward_alg类似，每个时间步存当前时间步每个tag对应之前的最优路径
        # 在已知发射分数和转移分数的情况下，求最优路径
        
        backpointers = [] # 回溯指针，回溯路径；backpointers[i][j]=第i帧到达j状态的所有路径中, 得分最高的那条在i-1帧是什么状态

        # Initialize the viterbi variables in log space
        # 初始化viterbi的forward_var变量
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            # 保存当前时间步的回溯指针
            bptrs_t = []  # holds the backpointers for this step
            # 保存当前时间步的viterbi变量
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size): # 对当前时间步各个tag循环，
               
                # 维特比算法记录最优路径时只考虑上一步的分数以及上一步tag转移到当前tag的转移分数
                # 不取决于当前tag的发射分数
                
                # 其他标签（B,I,E,Start,End）到标签next_tag的概率
                next_tag_var = forward_var + self.transitions[next_tag] # 每一步的最优路径+前一步转移到当前tag的transitions
                best_tag_id = argmax(next_tag_var) # 取最优路径（概率最大的一条的序号），当前tag由哪个之前tag转移来的
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1)) # 存的路径
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            # 更新forward_var，加上当前tag的发射分数feat
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            # 回溯指针 记录当前时间步各个tag来源前一步的tag
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        # 考虑转移到STOP_TAG的转移分数
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id] #最优路径对应的path_score

        # Follow the back pointers to decode the best path.
        # 通过回溯指针解码出最优路径
        best_path = [best_tag_id]
        # 从后向前走，找到一个best路径，best_path_id作为线头，反向遍历backpointers找到最优路径
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        # 去除START_TAG
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse() 
        return path_score, best_path # 最优路径对应的分数和最优路径

    def neg_log_likelihood(self, sentence, tags):# CRF损失函数，求负对数似然，作为loss
        # 由两部分组成，真实路径的分数和所有路径的总分数
        # 真实路径的分数应该是所有路径中分数最高的
        # log真实路径的分数/log所有可能路径的分数，越大越好，构造crf loss函数取反，loss越小越好
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats) # 所有可能的路径分数
        gold_score = self._score_sentence(feats, tags) # 真实路径的分数
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        # 通过BiLSTM提取发射分数
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        # 根据发射分数以及转移分数，通过viterbi解码找到一条最优路径
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

Run training



In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
# 构造训练数据
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
# 训练前检查模型预测结果
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        # 第一步，pytorch梯度累积，需要清零梯度
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        # 第二步，将输入转化为tensors
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        # 进行前向计算，取出crf loss
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        # 第四步，计算loss，梯度，通过optimizer更新参数
        loss.backward()
        optimizer.step()

# Check predictions after training
# 训练结束查看模型预测结果，对比观察模型是否学到
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))

    

Exercise: A new loss function for discriminative tagging
--------------------------------------------------------

It wasn't really necessary for us to create a computation graph when
doing decoding, since we do not backpropagate from the viterbi path
score. Since we have it anyway, try training the tagger where the loss
function is the difference between the Viterbi path score and the score
of the gold-standard path. It should be clear that this function is
non-negative and 0 when the predicted tag sequence is the correct tag
sequence. This is essentially *structured perceptron*.

This modification should be short, since Viterbi and score\_sentence are
already implemented. This is an example of the shape of the computation
graph *depending on the training instance*. Although I haven't tried
implementing this in a static toolkit, I imagine that it is possible but
much less straightforward.

Pick up some real data and do a comparison!


